<a href="https://colab.research.google.com/github/patricnilackshan/UoM_DMS_Toolkit/blob/main/UoM_DMS_Toolkit_by_Patric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# __UoM DMS Toolkit__ 💻

## **Introduction** 📋
UoM_DMS_Toolkit streamlines file tasks for students at UoM. Enjoy data-free downloads/uploads and multimedia processing, including Torrents, Youtube, M3U8 links. Say goodbye to bandwidth worries and focus on your education! 😁

<br>

### AN IDEA OF PATRIC NILACKSHAN 🧑‍💻 (pnilackshan@gmail.com)


<br>

<br>

## **Sign in to DMS** 🏫

In [ ]:
%%capture
!pip install requests yt-dlp ffmpeg-python libtorrent;
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!apt install python3-libtorrent


#@markdown <br><center><img src='https://dms.uom.lk/apps/theming/image/logo' height="200" alt="UoM-logo"/></center>
#@markdown <center><h3>Sign in</h3></center><br>


UserName = "UserName" # @param {type:"string"}
Password = "Password" # @param {type:"string"}
LoginDetail = f'"{UserName}:{Password}"'


# APIs
Header = '"OCS-APIRequest: true"'
UploadPoint = '"https://dms.uom.lk/remote.php/webdav/"'
SharePoint = '"https://dms.uom.lk/ocs/v2.php/apps/files_sharing/api/v1/shares?format=json"'

<br>

<br>

## **Download File from URL** 📥

In [ ]:
#@markdown <br><center><img src='https://dms.uom.lk/svg/core/actions/public?color=000' height="140" alt="Link-logo"/></center>
#@markdown <br><center><h3>Enter Download Link here...</h3></center><br>


Download_Link = "https://www.youtube.com/watch?v=AiD6SOOBKZI" # @param {type:"string"}
# @markdown <br>


# @markdown ### `FileName is required only for Torrents and Direct Download Links`
FileName = "Manasilayo" # @param {type:"string"}


import os
import sys
import time
import warnings
import subprocess

TorrentName = ""
downloaded_Torrent_Files = []


def download_torrent(Download_Link):
    import libtorrent as lt
    global FileName, TorrentName, downloaded_Torrent_Files
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=DeprecationWarning)
        ses = lt.session()
        params = {
            'save_path': './',
            'storage_mode': lt.storage_mode_t(2),
        }
        handle = lt.add_magnet_uri(ses, Download_Link, params)
        sys.stdout.write('Downloading metadata...\n')
        sys.stdout.flush()
        while not handle.has_metadata():
            time.sleep(1)
        TorrentName = handle.get_torrent_info().name()
        sys.stdout.write('Metadata downloaded, starting torrent download...\n')
        sys.stdout.write('Press Ctrl+C to stop\n')
        sys.stdout.flush()
        while handle.status().state != lt.torrent_status.seeding:
            s = handle.status()
            state_str = ['queued', 'checking', 'downloading metadata', 'downloading', 'finished', 'seeding', 'allocating', 'checking fastresume']
            progress = '%.2f%% complete (down: %.1f kB/s up: %.1f kB/s peers: %d) %s' % (
                s.progress * 100, s.download_rate / 1000, s.upload_rate / 1000, s.num_peers, state_str[s.state])
            sys.stdout.write('\r' + progress)
            sys.stdout.flush()
            time.sleep(1)
        downloaded_Torrent_Files = [file.path for file in handle.get_torrent_info().files()]
        sys.stdout.write('\nDownload complete!\n')
        sys.stdout.flush()
        print("\nDownloaded files:")
        for i, name in enumerate(downloaded_Torrent_Files):
          print(f"{i+1}: {name}")
        while True:
          try:
              file_selected = int(input("\n\nEnter 𝟎 to add all the downloaded files to a ZIP & Upload\n\n𝐎𝐑\n\nEnter the number corresponding to the file you want to upload to DMS: \n"))
              # ZIP all files
              if (file_selected == 0):
                print("\nZipping... Please wait...")
                command = ['zip','-r',f'{FileName}.zip',TorrentName]
                subprocess.run(command, check=True)
                FileName = f'{FileName}.zip'
                print(f'\n{FileName} selected for Upload')
                break
              # Select an individual file
              elif ((file_selected>0) and (file_selected<=len(downloaded_Torrent_Files))):
                file_selected = str(downloaded_Torrent_Files[file_selected-1])
                FileName = file_selected
                print(f'\n{FileName} selected for Upload')
                break
              else:
                  print("Please enter a valid number corresponding to the file you want to upload")
          except ValueError:
              print("Please enter a valid number corresponding to the file you want to upload")


def download_regular_file(Download_Link):
    import requests
    cwd = os.getcwd()
    full_output_path = os.path.join(cwd, FileName)
    response = requests.get(Download_Link, stream=True)
    if response.status_code == 200:
        with open(full_output_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=51200):
                if chunk:
                    f.write(chunk)
        print(f"File downloaded successfully and saved as {full_output_path}")
    else:
        print(f"Failed to download file from {Download_Link}. HTTP Status code: {response.status_code}")


def download_youtube_video(Download_Link):
    global FileName
    import yt_dlp
    # Set download options to merge the best video and audio streams
    ydl_opts = {
        'quiet': True,
        'outtmpl': f'%(title)s.%(ext)s',  # Output template for file name
        'merge_output_format': 'mp4',  # Merge into an mp4 file
        'postprocessors': [{
            'key': 'FFmpegVideoConvertor',
            'preferedformat': 'mp4',  # Convert to mp4 if necessary
        }]
    }
    # Extract video info without downloading
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(Download_Link, download=False)
    # List available formats
    formats = [f for f in info['formats'] if f.get('format_note') and f.get('ext')]
    for i, format in enumerate(formats):
        print(f"{i+1}. {format.get('format_note')} - {format.get('ext')}")
    # User prompt to select the format
    while True:
        try:
            choice = int(input("\nPlease enter a valid number corresponding to the file you want to upload: \n")) - 1
            print()
            if 0 <= choice < len(formats):
                ydl_opts['format'] = formats[choice]['format_id'] + '+bestaudio'
                with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                    info_dict = ydl.extract_info(Download_Link, download=True)
                    FileName = ydl.prepare_filename(info_dict)
                    print('\nVideo downloaded successfully!')
                break
            else:
                print("Please enter a valid number corresponding to the format you want to download")
        except ValueError:
            print("Please enter a valid number corresponding to the format you want to download")


def download_m3u8(Download_Link):
    command = [
        'ffmpeg',
        '-threads', '8',  # Use 8 threads
        '-i', Download_Link,   # Input file (m3u8 URL)
        '-c', 'copy',     # Copy the codec (fastest method)
        FileName          # Output file path
    ]
    subprocess.run(command, check=True)


def main():
    if Download_Link.startswith('magnet:?xt=urn:btih:'):
        download_torrent(Download_Link)
    elif Download_Link.endswith('.m3u8'):
        download_m3u8(Download_Link)
    elif (('youtube.com' in Download_Link) or ('youtu.be' in Download_Link)):
        download_youtube_video(Download_Link)
    else:
        download_regular_file(Download_Link)


main()

<br>

<br>

## **Upload File to DMS** 📤

In [ ]:
#@markdown <br><center><img src='https://dms.uom.lk/apps/circles/img/black_circle.svg' height="140" alt="Upload-logo"/></center>
#@markdown <br><center><h3>Upload to DMS</h3></center><br>


import re
import sys
import subprocess

def execute_curl_with_progress(curl_command):
    # Start the curl command in a subprocess with stdout redirected
    process = subprocess.Popen(curl_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
    try:
        # Initialize variables to store the previous progress
        prev_progress = 0
        # Loop through the output of the command
        for line in process.stderr:
            # Extract progress information using regex
            match = re.search(r'(\d+(\.\d+)?)%', line)
            if match:
                progress = float(match.group(1))
                # Calculate the number of characters to represent the progress bar
                bar_length = 50
                filled_length = int(bar_length * progress / 100)
                filled_bar = '█' * filled_length  # Use '█' for the filled part
                empty_bar = '░' * (bar_length - filled_length)  # Use '░' for the empty part
                # Display progress information
                sys.stdout.write(f"\rUpload Progress: [{filled_bar}{empty_bar}] {progress:.1f}%")
                sys.stdout.flush()
                # Update previous progress
                prev_progress = progress
            else:
                # Skip lines that don't contain progress updates
                continue
        # Wait for the process to finish and get the return code
        return_code = process.wait()
        # Check if the command was successful
        if return_code == 0:
            print("\nFile uploaded to DMS successfully.")
        else:
            print("\nError occurred during file upload.")
    except Exception as e:
        print("\nAn error occurred:", e)


# Construct the curl command with --progress-bar option
curl_command = f'curl -u {LoginDetail} --progress-bar -T "{FileName}" "{UploadPoint}"'


# Execute the curl command and show progress
execute_curl_with_progress(curl_command)

<br>

<br>

## **Share File from the DMS** 🔁

In [ ]:
#@markdown <br><center><img src='https://dms.uom.lk/svg/core/actions/share?color=000' height="140" alt="Share-logo"/></center>
#@markdown <br><center><h3>Share File</h3></center><br>


import json
permissionBody = str('"'+'path='+FileName+'&shareType=3&permissions=1'+'"')
json_response = !curl -u {LoginDetail} -X POST -d {permissionBody} {SharePoint} -H {Header}
json_string = ''.join(json_response)
share = json.loads(json_string)['ocs']['data']
try:
  print(f"Share ID : {share['id']}\nFile Path : {share['path']}\nDMS Link : {share['url']}\n")
except:
  print("Error during Sharing")

<br>

<br>

## **Get details of all shares in DMS Account** 📢

In [ ]:
#@markdown <br><center><img src='https://dms.uom.lk/svg/core/actions/tag?color=000' height="140" alt="Info-logo"/></center>
#@markdown <br><center><h3>Get Share Details</h3></center><br>


import json
json_response = !curl -u {LoginDetail} -X GET {SharePoint} -H {Header}

# Convert the SList to a string and then parse it as JSON
json_string = ''.join(json_response)
json_data = json.loads(json_string)["ocs"]["data"]

# Print details of shares
if (len(json_data) > 0):
  for share in json_data:
    print(f"Share ID : {share['id']}\nFile Path : {share['path']}\nDMS Link : {share['url']}\n")
else:
  print("No shares found for this DMS Account.")

<br>

<br>

## **Delete a share with its Share_ID** 🗑️

In [ ]:
#@markdown <br><center><img src='https://dms.uom.lk/core/img/actions/delete.svg' height="140" alt="Delete-logo"/></center>
#@markdown <br><center><h3>Delete a share</h3></center><br>


Share_ID = 289018 # @param {type:"integer"}
deletePoint = str(f'{SharePoint[:-13]}/{Share_ID}?format=json"')
json_response = !curl -u {LoginDetail} -X DELETE {deletePoint} -H {Header}
json_string = ''.join(json_response)
try:
  statuscode = json.loads(json_string)["ocs"]["meta"]["statuscode"]
  if (statuscode == 200):
    print(f"Share with Share ID: {Share_ID} deleted successfully.")
  else:
    print("Error during deletion")
except:
  print("Error during deletion")

<br>

<br>

## **Mount DMS WebDAV in My Computer** 🖴

### To map DMS as a local disk in Windows 🪟, run this command in Command Prompt
`
net use Z: https://dms.uom.lk/remote.php/webdav/ /user:<userName> <userPassword>
`

Replace 'userName' and 'userPassword' with actual values

<br>

### To access DMS in Linux 🐧, enter this address in the `Enter server address` field in your File Manager
`
davs://dms.uom.lk/remote.php/webdav/
`
<br>

<br>

<br>

## **Conclusion** 😎

### This notebook provides a convenient way to interact with the DMS for educational purposes, enabling data-free downloads, uploads, and sharing of files and folders. 🧑‍💻

<br>

### © PATRIC NILACKSHAN (pnilackshan@gmail.com)